In [2]:
import urllib.request
import pandas as pd
import requests
import io
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [3]:
hsb2 = pd.read_csv("C:\\Users\\Admin\\Desktop\\Vaishnavi\\Auto\\DriverData1.csv")

In [4]:
hsb2.head()

,ID,SubVehID,Choice,Gap,RelativeSpeed,TW
0,1,9,2,12.0530,-0.927,0
1,2,13,1,16.3285,2.448,1
2,3,13,3,15.6910,1.845,1
3,4,13,1,15.0280,0.999,1
4,5,36,2,12.9030,0.144,0


In [5]:
hsb2.dtypes

ID                 int64
SubVehID           int64
Choice             int64
Gap              float64
RelativeSpeed    float64
TW                 int64
dtype: object

In [ ]:
hsb2["Gap"] = hsb2["Gap"].astype('int64')
hsb2["RelativeSpeed"] = hsb2["RelativeSpeed"].astype('int64')
hsb2["TW"] = hsb2["TW"].astype('category')

In [ ]:
hsb2.dtypes

In [ ]:
hsb3 = hsb2


In [4]:
hsb3.head()

NameError: name 'hsb3' is not defined

In [16]:
y = hsb3['Choice']
X = hsb3.drop(['ID','SubVehID','Choice'],axis=1)

In [17]:
import pandas as pd
import statsmodels.api as sm
from scipy import stats

X = sm.add_constant(X)

# Fit separate binary logistic regression models for each category against all others
unique_choices = y.unique()
coefficients = {}

for choice in unique_choices:
    y_binary = (y == choice).astype(int)
    model = sm.Logit(y_binary, X)
    result = model.fit(disp=False)
    coefficients[choice] = result.params

# Combine coefficients into a single DataFrame
coefficients_df = pd.DataFrame(coefficients).T
coefficients_df.columns = ['Intercept'] + list(X.columns[1:])

print(coefficients_df)

X_null = sm.add_constant(pd.DataFrame({'Intercept': [1] * len(y)}))
null_coefficients = {}

for choice in unique_choices:
    y_null_binary = (y == choice).astype(int)
    null_model = sm.Logit(y_null_binary, X_null)
    null_result = null_model.fit(disp=False)
    null_coefficients[choice] = null_result.params

# Combine coefficients into a single DataFrame
coefficients_df = pd.DataFrame(null_coefficients).T
coefficients_df.columns = ['Intercept'] + list(X_null.columns[1:])

LL_model = result.llf  # Log-likelihood of the fitted model
LL_null = null_result.llf  # Log-likelihood of the null model
chi_square_stat = 2 * (LL_model - LL_null)
df_diff = result.df_model - null_result.df_model
p_value = stats.chi2.sf(chi_square_stat, df_diff)
print("Chi-square statistic:", chi_square_stat)
print("Degrees of freedom:", df_diff)
print("p-value:", p_value)


   Intercept       Gap  RelativeSpeed        TW
2  -1.775865 -0.011993      -0.082317  0.030870
1   2.821543 -0.229066      -0.380298  0.281062
3  -3.726033  0.249797       0.459104 -0.343074
Chi-square statistic: 547.5862685536517
Degrees of freedom: 3.0
p-value: 2.3179810563816417e-118


In [12]:


X = sm.add_constant(X)

# Fit separate binary logistic regression models for each category against all others
unique_choices = y.unique()
coefficients = {}

for choice in unique_choices:
    y_binary = (y == choice).astype(int)
    model = sm.Logit(y_binary, X)
    result = model.fit(disp=False)
    coefficients[choice] = result.params

# Combine coefficients into a single DataFrame
coefficients_df = pd.DataFrame(coefficients).T
coefficients_df.columns = ['Intercept'] + list(X.columns[1:])

print(coefficients_df)

X_null = sm.add_constant(pd.DataFrame({'Intercept': [1] * len(y)}))
null_coefficients = {}

for choice in unique_choices:
    y_null_binary = (y == choice).astype(int)
    null_model = sm.Logit(y_null_binary, X_null)
    null_result = null_model.fit(disp=False)
    null_coefficients[choice] = null_result.params

# Combine coefficients into a single DataFrame
coefficients_df = pd.DataFrame(null_coefficients).T
coefficients_df.columns = ['Intercept'] + list(X_null.columns[1:])

# Calculate McFadden's pseudo-R^2
LL_model = result.llf  # Log-likelihood of the fitted model
LL_null = null_result.llf  # Log-likelihood of the null model
rho_squared = 1 - (LL_model / LL_null)

# Print the result
print("McFadden's pseudo-R^2:", rho_squared)


   Intercept       Gap  RelativeSpeed        TW
2  -1.775865 -0.011993      -0.082317  0.030870
1   2.821543 -0.229066      -0.380298  0.281062
3  -3.726033  0.249797       0.459104 -0.343074
McFadden's pseudo-R^2: 0.16519589484174013


In [10]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



# Create a multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# Fit the model to the data
model.fit(X, y)

# Predict the class labels for the same data
y_pred = model.predict(X)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)
print(model.intercept_)
print(model.coef_)

Accuracy: 0.64875
[ 2.14744897 -0.45742293 -1.69002604]
[[-0.12946688 -0.20774494  0.15692087]
 [-0.02023765 -0.07849998  0.0538605 ]
 [ 0.14970453  0.28624493 -0.21078137]]


In [33]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

# Create a multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                           C=1.0, max_iter=1000000)



# Fit the model to the data
model.fit(X, y)

# Get the log-likelihood of the fitted model
y_prob = model.predict_proba(X)
LL_model = -log_loss(y, y_prob, normalize=False)

# Fit the null model (only an intercept)
null_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                                C=1.0, max_iter=1000000, fit_intercept=True)
X_null = np.ones((X.shape[0], 1))  # Only intercept
null_model.fit(X_null, y)

# Get the log-likelihood of the null model
y_null_prob = null_model.predict_proba(X_null)
LL_null = -log_loss(y, y_null_prob, normalize=False)

# Compute rho-squared
rho_squared = 1 - (LL_model / LL_null)
print("Rho-squared:", rho_squared)


Rho-squared: 0.12235023609532403


In [34]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Compute mean of cross-validation scores
mean_cv_score = cv_scores.mean()
print("Mean Cross-Validation Score:", mean_cv_score)


Mean Cross-Validation Score: 0.63375


In [14]:
# define the multinomial logistic regression model with a default penalty
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                           C=1.0, max_iter = 1000000)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Accuracy: 0.647 (0.025)


In [11]:
result = model.fit(X, y)

In [12]:
print(result.intercept_)
print(result.coef_)

[ 2.14744897 -0.45742293 -1.69002604]
[[-0.12946688 -0.20774494  0.15692087]
 [-0.02023765 -0.07849998  0.0538605 ]
 [ 0.14970453  0.28624493 -0.21078137]]


In [13]:
summary = pd.DataFrame(zip(X.columns, np.transpose(result.coef_.tolist()[0])), 
                       columns=['features', 'coef'])

In [14]:
print(summary)

        features      coef
0            Gap -0.129467
1  RelativeSpeed -0.207745
2             TW  0.156921


In [19]:
p1= []
p2 = []
p3 = []
# Initialize an empty list to store predicted probabilities
for i in range(len(hsb2.ID)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    yhat = model.predict_proba(row)
    # Summarize the predicted probabilities and append to the list
    p1.append(yhat[0][0])
    p2.append(yhat[0][1])
    p3.append(yhat[0][2])

hsb2['P1'] = p1
hsb2['P2'] = p2
hsb2['P3'] = p3




    

In [20]:
# predict the class label
yhat = model.predict(row)
# summarize the predicted class
print('Predicted Class: %d' % yhat[0])

c = []
# Initialize an empty list to store predicted probabilities
for i in range(len(hsb2.ID)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    yhat = model.predict(row)
    # Summarize the predicted probabilities and append to the list
    c.append(yhat[0])
    

hsb2['Predicted'] = c

Predicted Class: 3


In [21]:
print(hsb2.drop(['SubVehID'],axis=1))

        ID  Choice  Gap  RelativeSpeed TW        P1        P2        P3  \
0        1       2   12              0  0  0.529563  0.145174  0.325263   
1        2       1   16              2  1  0.200624  0.099450  0.699926   
2        3       3   15              1  1  0.333248  0.130143  0.536609   
3        4       1   15              0  1  0.429973  0.147559  0.422469   
4        5       2   12              0  0  0.529563  0.145174  0.325263   
...    ...     ...  ...            ... ..       ...       ...       ...   
2395  2396       1   13              1  1  0.447387  0.140431  0.412182   
2396  2397       3   13              1  1  0.447387  0.140431  0.412182   
2397  2398       1   13              0  1  0.544068  0.150073  0.305859   
2398  2399       3   15              3  1  0.171511  0.086738  0.741751   
2399  2400       2   15              4  1  0.115431  0.066431  0.818138   

      Predicted  
0             1  
1             3  
2             3  
3             1  
4        

In [30]:
pip install scipy


In [45]:
# importing packages
import scipy.stats as stats 
import numpy as np 
  
# no of hours a student studies 
# in a week vs expected no of hours 
observed_data = hsb2.Choice
expected_data = hsb2.Predicted 
  
# Chi-Square Goodness of Fit Test 
chi_square_test_statistic, p_value = stats.chisquare( 
    observed_data, expected_data) 
  
# chi square test statistic and p value 
print('chi_square_test_statistic is : ' +
      str(chi_square_test_statistic))  # Corrected the print statement
  
# importing packages 
import scipy.stats as stats 
import numpy as np 
  
# no of hours a student studies 
# in a week vs expected no of hours 
observed_data = hsb2.Choice
expected_data = hsb2.Predicted 
  
  
# Chi-Square Goodness of Fit Test 
chi_square_test_statistic, p_value = stats.chisquare( 
    observed_data, expected_data) 
  
# chi square test statistic and p value 
print('chi_square_test_statistic is : ' +
      str(chi_square_test_statistic)) 
print('p_value : ' + str(p_value)) 
  
  
# find Chi-Square critical value 
print(stats.chi2.ppf(1-0.05, df=3))  # Corrected the print statement


ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
0.01763632677883641

In [42]:
print(len(hsb2.Choice)-len(hsb2.Predicted))

0


In [46]:
X = sm.add_constant(X)


      const  Gap  RelativeSpeed TW
0       1.0   12              0  0
1       1.0   16              2  1
2       1.0   15              1  1
3       1.0   15              0  1
4       1.0   12              0  0
...     ...  ...            ... ..
2395    1.0   13              1  1
2396    1.0   13              1  1
2397    1.0   13              0  1
2398    1.0   15              3  1
2399    1.0   15              4  1

[2400 rows x 4 columns]


In [47]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import MNLogit



X = sm.add_constant(X)



# Fit the multinomial logistic regression model
model = MNLogit(y, X)
result = model.fit()

# Get the log-likelihood of the fitted model
log_likelihood_fitted = result.llf

# Get the log-likelihood of the null model
# Null model is the model with only the intercept
X_null = sm.add_constant(pd.DataFrame({'intercept': np.ones(len(y))}))
null_model = MNLogit(y, X_null)
null_result = null_model.fit()
log_likelihood_null = null_result.llf

# Calculate the rho-squared statistic
rho_squared = 1 - (log_likelihood_fitted / log_likelihood_null)
print('Rho-squared: ', rho_squared)


Optimization terminated successfully.
         Current function value: 0.863698
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.984104
         Iterations 6
Rho-squared:  0.1223504172765787


In [49]:
# Example dataset

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fitting the MNLogit model
model = sm.MNLogit(y, X)
result = model.fit()

# Display the summary of the model
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.863698
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                 Choice   No. Observations:                 2400
Model:                        MNLogit   Df Residuals:                     2392
Method:                           MLE   Df Model:                            6
Date:                Thu, 06 Jun 2024   Pseudo R-squ.:                  0.1224
Time:                        21:51:09   Log-Likelihood:                -2072.9
converged:                       True   LL-Null:                       -2361.8
Covariance Type:            nonrobust   LLR p-value:                1.331e-121
     Choice=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -2.6055      0.604     -4.311      0.000      -3.790      -1.421
Gap               0.

In [50]:
model = LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')

In [ ]:
result = model.fit(X, y)